<a href="https://colab.research.google.com/github/yudumpacin/AISummerCamp/blob/main/DatasetOperations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers xformers trl datasets wandb bitsandbytes peft --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
!pip install accelerate --quiet

In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [19]:
from datasets import load_dataset,DatasetDict,Dataset,concatenate_datasets

In [5]:
from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [7]:
#cache_dir=''#google drive as cashe_dir??
model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'

max_seq_length=8192

In [8]:
#load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=max_seq_length,
    dtype=None,#float 16 for Tesla T4,V100,BLoat16 for Ampere
    load_in_4bit=True,#Use 4 bit quantization to reduce memory, to load bf16 set False
    #cache_dir=cache_dir,#model localde kaydolacaksa comment
)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
base_dataset_name = "Yudum/wiki-tr-50000"

In [ ]:
"""
dataset = load_dataset('musabg/wikipedia-tr', split='train')
total_rows = len(dataset)
print(f'Total rows in the dataset: {total_rows}')

# Rastgele 50000 satır seç
sample_size = 50000
random_indices = random.sample(range(total_rows), sample_size)
sampled_dataset = dataset.select(random_indices)


print(f'Sampled dataset size: {len(sampled_dataset)}')
data = {
    "title": sampled_dataset['title'],
    "text": sampled_dataset['text']
}

new_dataset = DatasetDict({"train": Dataset.from_dict(data)})
new_dataset.push_to_hub("Yudum/wiki-tr-50000")
"""

In [39]:
blend_dataset_name = "mlabonne/orpo-dpo-mix-40k"

In [11]:
base_dataset = load_dataset(base_dataset_name,split='train')

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [22]:
def merge_columns(example):
    example['text'] = f"{example['title']}. {example['text']}"
    return example

base_dataset = base_dataset.map(merge_columns)
base_dataset = base_dataset.remove_columns([col for col in base_dataset.column_names if col!='text'])

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [27]:
base_dataset["text"][0]

"Prenses Martha Sahili. Prenses Martha Sahili, Antarktika'da Kraliçe Maud Toprakları'nda bulunan bir kıyıdır. Kıyı, Stancomb-Wills Buzulu'nun son kısmı olup tüm kıyı şeridi 20 ila 35 m yüksekliğinde buz kayalıklarına sahip buz sahanlıkları sınırlanmıştır.\n\nSahil, Antarktika anakarasının 1820'de Fabian von Bellingshausen ve Mihail Lazarev tarafından keşfedilen ilk bölümüdür. Adını, Norveç Prensesi Märtha'dan almaktadır.\n\nKaynakça \n M. Messick, John F. Splettstoesser, Peter Harrison and Maria Gavrilo, New breeding site for emperor penguins (Aptenodytes forsteri) on Princess Martha Coast, Queen Maud Land, East Antarctica , Volume 35, Issue 193 April 1999 , pp.\xa0156–157\n\nAntarktika kıyıları\nKraliçe Maud Toprakları"

In [28]:
blend_size = int(0.05*len(base_dataset)/(1-0.05))

In [55]:
blend_dataset = load_dataset(blend_dataset_name,split=f'train[:{blend_size}]')

In [56]:
blend_dataset

Dataset({
    features: ['source', 'chosen', 'rejected', 'prompt', 'question'],
    num_rows: 2631
})

In [57]:
blend_dataset = blend_dataset.filter(lambda r: r["source"] != "toxic-dpo-v0.2")

In [65]:
tokenizer.apply_chat_template(blend_dataset['chosen'][0], tokenize=False)

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nThe setting is an otherworldly, yet eerily familiar, metropolis known as "Zephyria." It\'s a city suspended in the ether, floating amidst nebulous clouds of cosmic dust. The architecture here is surreal and alien, with buildings that twist and spiral like strands of DNA, reaching towards the cosmos. Streets are paved with luminescent cobblestones, casting soft hues of blues and purples, reflecting off iridescent structures. Strange vegetation, vibrant and bioluminescent, thrives everywhere, creating a stark contrast against the deep indigo sky.\n\nNow, immerse me in this ethereal journey through Zephyria.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAs you step onto the teleportation platform, there\'s a momentary sense of disorientation before your surroundings change abruptly. You find yourself standing on the outskirts of Zephyria, gazing at the sprawling metropolis that glows softly under the starlit canvas 

In [66]:
def apply_chat_template_(example):
  example['text'] = tokenizer.apply_chat_template(example['chosen'], tokenize=False)
  return example

In [67]:
blend_dataset = blend_dataset.map(apply_chat_template_)

Map:   0%|          | 0/2631 [00:00<?, ? examples/s]

In [69]:
blend_dataset["text"][0]

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nThe setting is an otherworldly, yet eerily familiar, metropolis known as "Zephyria." It\'s a city suspended in the ether, floating amidst nebulous clouds of cosmic dust. The architecture here is surreal and alien, with buildings that twist and spiral like strands of DNA, reaching towards the cosmos. Streets are paved with luminescent cobblestones, casting soft hues of blues and purples, reflecting off iridescent structures. Strange vegetation, vibrant and bioluminescent, thrives everywhere, creating a stark contrast against the deep indigo sky.\n\nNow, immerse me in this ethereal journey through Zephyria.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAs you step onto the teleportation platform, there\'s a momentary sense of disorientation before your surroundings change abruptly. You find yourself standing on the outskirts of Zephyria, gazing at the sprawling metropolis that glows softly under the starlit canvas 

In [70]:
blend_dataset = blend_dataset.remove_columns([col for col in blend_dataset.column_names if col != 'text'])

In [71]:
blend_dataset

Dataset({
    features: ['text'],
    num_rows: 2631
})

In [72]:
merged_dataset_2 = concatenate_datasets([base_dataset, blend_dataset])

In [73]:
merged_dataset_2

Dataset({
    features: ['text'],
    num_rows: 52631
})

In [74]:
train_test_split_2 = merged_dataset_2.train_test_split(test_size=0.05)

In [76]:
final_dataset_2 = DatasetDict({
    "train":train_test_split_2["train"],
    "test":train_test_split_2["test"]
})

In [77]:
final_dataset_2.push_to_hub("Yudum/wiki-tr-orpo-dpo-mix")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/50 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Yudum/wiki-tr-orpo-dpo-mix/commit/ea17a09e08160adbd20cfbf2f595e97363f376fb', commit_message='Upload dataset', commit_description='', oid='ea17a09e08160adbd20cfbf2f595e97363f376fb', pr_url=None, pr_revision=None, pr_num=None)